<a href="https://colab.research.google.com/github/anupambcharya/MachineLearning/blob/main/NLP-Classification/MyEmailClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/anupambcharya/machinelearning.git

Cloning into 'machinelearning'...
remote: Enumerating objects: 843, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 843 (delta 28), reused 4 (delta 4), pack-reused 791 (from 2)
Receiving objects: 100% (843/843), 19.75 MiB | 27.37 MiB/s, done.
Resolving deltas: 100% (193/193), done.


In [ ]:



import nltk
>>> nltk.download('stopwords')
>>> nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import pickle as cPickle

In [ ]:
# Function to extract summary of files
def getSummary(text, num_sentences=3):
    stop_words = set(stopwords.words("english"))
    words = word_tokenize(text)

    # Calculate word frequency
    word_frequencies = {}
    for word in words:
        word = word.lower()
        if word not in stop_words and word.isalpha():
            if word not in word_frequencies:
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1

    # Score sentences based on word frequency
    sentences = sent_tokenize(text)
    sentence_scores = {}
    for sentence in sentences:
        for word in word_tokenize(sentence.lower()):
            if word in word_frequencies:
                if sentence not in sentence_scores:
                    sentence_scores[sentence] = word_frequencies[word]
                else:
                    sentence_scores[sentence] += word_frequencies[word]

    # Select top sentences
    summary_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]
    summary = ' '.join(summary_sentences)
    return summary

In [ ]:
# Define the directory containing your text files
mainFolderPath = "machinelearning/NLP-Classification/Data/Training"

# Initialize a feature extractor (e.g., TF-IDF Vectorizer)
vectorizer = TfidfVectorizer()

# List to store extracted features for each file
mailFeatures = []
mailCategory = []

In [ ]:
# Loop through each file in the directory
for folderName in os.listdir(mainFolderPath):
    #print(folderName)
    subFolder = mainFolderPath + "/" + folderName

    for fileName in os.listdir(subFolder):
            if fileName.endswith(".txt"):  # Process only text files
                filepath = os.path.join(subFolder, fileName)
                with open(filepath, 'r', encoding='utf-8') as file:
                    # Get full content
                    txtContent = file.read()
                    # Get summary of full content
                    txtContent = getSummary(txtContent, num_sentences=3)

                # Perform feature extraction on the text content/mail content
                # Here, we're are performing a per-file operation
                # to get TF-IDF features against the summary of each mail (stored in text file):

                #txtFeature = vectorizer.fit_transform([txtContent]) # Fit and transform on single document
                #print(txtContent)
                # Store the features and Mail Category
                mailFeatures.append(txtContent)
                mailCategory.append(folderName)


data = {'mailFeature': mailFeatures, 'mailCategory':mailCategory}
df = pd.DataFrame(data)

In [ ]:
# 2. Feature Extraction (TF-IDF)
# TF-IDF converts text into numerical feature vectors
tfidf_vectorizer = TfidfVectorizer(encoding='utf-8', lowercase=True, analyzer='word')
X = tfidf_vectorizer.fit_transform(df['mailFeature'])
#print(df['mailCategory'])
y = df['mailCategory']

# 3. Split Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 4. Model Training (Multinomial Naive Bayes)
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB()

In [ ]:
f = open("classifier.cPickle", "wb")
f.write(cPickle.dumps(classifier))
f.close()

In [1]:
# 5. Prediction and Evaluation
y_pred = classifier.predict(X_test)

# Example of predicting new text
#new_text = ["Hi! Saw Panchayat last night. The character of Vinod was outstanding. This web-series was truly amazing! You must watch it"]
new_text = ["""Hello all Residents, This is to bring to your notice that for past few weeks there has a substantial increase in incidents of
             car theif in our colony. Last week, there were 3 instances of car theif and vandanlisation in A and E block. While we are picking
             this with the Police and local authority, we would like to urge you to do your bit and take ample care of your belongings especially
             cars that are parked outside of our gate.

              We hope that the local Police will take these incidents seriously and increase colony vigilance at night.

              Will keep you posted in this regard.

              Thank you,
              President,
              RWA"""]
new_text_vectorized = tfidf_vectorizer.transform(new_text)
prediction = classifier.predict(new_text_vectorized)
print(f"\nPrediction for Email with content : '{new_text[0]}':\n The e-mail goes to folder: {prediction[0]}")

SyntaxError: unterminated string literal (detected at line 6) (ipython-input-1-3530038262.py, line 6)